In [ ]:
import pandas as pd

df = pd.DataFrame()

In [ ]:
image_path = "../../data/images/City/image_8.jpg"

In [ ]:
from PIL import Image
image = Image.open(image_path)
image = image.convert("RGB")
image.save("reference_map.png", "PNG")

In [ ]:
import base64
from openai import OpenAI
client = OpenAI()

prompt = "Create a top-down image similar to this one, rearrange the components from the image and make them simpler so that they are easily identified by vision models."
result = client.images.edit(
    model="gpt-image-1",
    image=open("reference_map.png", "rb"),
    prompt=prompt
)

image_base64 = result.data[0].b64_json
image_bytes = base64.b64decode(image_base64)

with open("new_map.png", "wb") as f:
    f.write(image_bytes)

In [ ]:
import base64
import csv
import os
from pathlib import Path
from openai import OpenAI

def process_images_from_csv(csv_file_path, output_folder="simplified_images"):
    """
    Process images listed in a CSV file and create simplified versions.
    
    Args:
        csv_file_path (str): Path to the CSV file containing image paths
        output_folder (str): Name of the output folder for simplified images
    """

    client     = OpenAI()
    prompt     = "Create a top-down image similar to this one, rearrange the components from the image and make them simpler so that they are easily identified by vision models."    
    output_dir = Path(output_folder)
    output_dir.mkdir(exist_ok=True)
    
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        error_count = 0
        processed_count = 0
        
        for row_num, row in enumerate(reader, start=1):
            if row_num == 1:
                continue
            if not row or not row[0].strip():
                continue
            
            image_path = row[0].strip()
            if "Pasture" not in image_path:
                continue
            print(f"{image_path}")
            try:

                if not os.path.exists(image_path):
                    print(f"Warning: File not found - {image_path}")
                    error_count += 1
                    continue
                
                input_file = Path(image_path)
                subfolder  = input_file.parent.name
                output_d   = Path(output_dir / subfolder)
                output_d.mkdir(exist_ok=True)
                
                output_filename = f"{subfolder}/{input_file.stem}.jpg"
                output_path     = output_dir / output_filename
                print(f"Processing {image_path} -> {output_path}")
                result = client.images.edit(
                    model  = "gpt-image-1",
                    image  = open(image_path, "rb"),
                    prompt = prompt
                )

        
                image_base64    = result.data[0].b64_json
                image_bytes     = base64.b64decode(image_base64)


                with open(output_path, "wb") as f:
                    f.write(image_bytes)
                
                print(f"✓ Saved: {output_path}")
                processed_count += 1
                
            except Exception as e:
                error_count += 1
                continue
    
    print(f"\nProcessing complete!")
    print(f"Successfully processed: {processed_count} images")
    print(f"Errors encountered: {error_count} images")
    print(f"Output folder: {output_dir.absolute()}")


if __name__ == "__main__":

    csv_file = "../../data/output/image_objects.csv"
    process_images_from_csv(csv_file, "simplified_images")

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import base64

load_dotenv("../../.env")
client        = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
list_objects  = ["House", "Fortress", "Barrel", "Bridge", "Church", "Fountain", "Tree", "Hut", "Tent", "Windmill", "Cannon", "Well", "Lighthouse", "Farm", "Bonfire", "Table", "Boat", "Chest", "Spikes", "Statue", "Wagon", "Sign", "Lantern", "Cage", "Weapon", "Ladder", "Crate", "Cauldron", "Anvil", "Store"]
list_terrains = ["Snow", "Ice", "Water", "Dirt", "Grass", "Rock", "Lava", "Swamp", "Sand"]

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def get_image_objects(image_path):

    try:
        base64_image = encode_image(image_path)
        completion   = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        { "type": "text", "text":
                            f"List all the objects present in this image out of the following list: {list_objects}. Also, name all the types of terrain found out of this list: {list_terrains}. Lastly, provide a detailed description of the positional layout of the scene, which is a fantasy map for a Dungeons and Dragons game. Provide your answer separated by semi-colons and list items separated by commas, like this: 'objects: X, Y, Z; terrains: K, I, J; description: lorem ipsum. Be concise while describing the scene. Solely provide the information in the requested format and do not include any other information.'"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                            },
                        },
                    ],
                }
            ],
        )

        return completion.choices[0].message.content
    except Exception as e:
        print(e)
        return None

In [ ]:
for root, dirs, files in os.walk("../../data/images"):
    
    for file in files:

        if not file.endswith(".jpg"):
            continue
        
        image_path  = os.path.join(root, file)
        text_output = get_image_objects(image_path)
        
        try:
            objects     = text_output.split(";")[0].split(":")[1].strip().split(",")
            terrains    = text_output.split(";")[1].split(":")[1].strip().split(",")
            description = text_output.split(";")[2].split(":")[1].strip()

            df          = df.append({
                "path"       : image_path,
                "objects"    : objects,
                "terrains"   : terrains,
                "description": description,
                "plain_text" : text_output

            }, ignore_index=True)
        except:
            print(f"Error processing image: {image_path}, text output: {text_output}")

In [ ]:
df.to_csv("../../data/output/scene_descriptions.csv", index=False)